# Importing Libraries

In [23]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import joblib  # For saving non-PyTorch models

In [5]:
# Ensure 'models' directory exists
os.makedirs("models", exist_ok=True)

In [20]:
# Function to load dataset based on CNN model type and dataset type (train/test)
def load_dataset(cnn_model="ResNet-50", data_type="train"):
    base_path = os.path.join("features", cnn_model, data_type)  # Example: features/resnet

    # File names
    X_filename = f"hff_features_{data_type}.npy"
    y_filename = f"hff_labels_{data_type}.npy"

    X_path = os.path.join(base_path, X_filename)
    y_path = os.path.join(base_path, y_filename)

    # Load Features and Labels
    if not os.path.exists(X_path) or not os.path.exists(y_path):
        raise FileNotFoundError(f"❌ Dataset files not found in {base_path}")

    X = np.load(X_path)
    y = np.load(y_path)

    print(f"✅ Loaded {data_type} data from {cnn_model} features: {X.shape}, Labels: {y.shape}")
    
    return X, y

In [14]:
# Define the Neural Network Classifier
class HFFClassifier(nn.Module):
    def __init__(self, input_size, num_classes):
        super(HFFClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.softmax(self.fc3(x))
        return x

In [15]:
# Training Function for Neural Network
def train_neural_network(cnn_model="resnet", num_epochs=30, batch_size=64, optimizer_type="adam"):
    print(f"🔄 Loading dataset from {cnn_model} features...")
    X_train, y_train = load_dataset(cnn_model, "train")
    X_test, y_test = load_dataset(cnn_model, "test")
    
    # Convert to PyTorch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    input_size = X_train.shape[1]
    num_classes = len(set(y_train))
    model = HFFClassifier(input_size, num_classes)

    # Choose Optimizer
    if optimizer_type.lower() == "adam":
        optimizer = optim.Adam(model.parameters(), lr=0.001)
    elif optimizer_type.lower() == "sgd":
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    else:
        raise ValueError("Unsupported optimizer. Use 'adam' or 'sgd'.")

    criterion = nn.CrossEntropyLoss()

    # Training Loop
    for epoch in range(num_epochs):
        model.train()
        total_loss, correct_train, total_train = 0, 0, 0

        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct_train += (predicted == targets).sum().item()
            total_train += targets.size(0)

        train_accuracy = correct_train / total_train * 100

        # Evaluation Phase
        model.eval()
        correct_test, total_test = 0, 0
        with torch.no_grad():
            for inputs, targets in test_loader:
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                correct_test += (predicted == targets).sum().item()
                total_test += targets.size(0)

        test_accuracy = correct_test / total_test * 100
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}, "
              f"Train Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%")

    print("✅ Training Completed!")
    
    # Ensure models directory exists
    os.makedirs("models", exist_ok=True)

    model_path = f"models/hff_nn_{cnn_model}.pth"
    torch.save(model.state_dict(), model_path)
    print(f"✅ Neural Network Model Saved: {model_path}")

    return model

In [ ]:
# Training Function for Other Models
def train_other_models(cnn_model="resnet", model_type="random_forest"):
    print(f"🔄 Loading dataset from {cnn_model} features...")
    X_train, y_train = load_dataset(cnn_model, "train")
    X_test, y_test = load_dataset(cnn_model, "test")

    # Initialize model
    if model_type.lower() == "random_forest":
        model = RandomForestClassifier(n_estimators=100, random_state=42)
    elif model_type.lower() == "svm":
        model = SVC(kernel="rbf", probability=True)
    elif model_type.lower() == "logistic_regression":
        model = LogisticRegression(max_iter=1000)
    else:
        raise ValueError("Unsupported model. Choose 'random_forest', 'svm', 'logistic_regression', or 'gradient_boosting'.")

    print(f"🔄 Training {model_type} model on {cnn_model} features...")
    model.fit(X_train, y_train)

    # Evaluate Training Accuracy
    y_train_pred = model.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_train_pred)

    # Evaluate Test Accuracy
    y_test_pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_test_pred)

    print(f"✅ {model_type} Model Accuracy on {cnn_model}:")
    print(f"   - Train Accuracy: {train_accuracy * 100:.2f}%")
    print(f"   - Test Accuracy: {test_accuracy * 100:.2f}%")

    # Ensure models directory exists
    os.makedirs("models", exist_ok=True)

    # Save Model
    model_path = f"models/hff_{model_type}_{cnn_model}.pkl"
    joblib.dump(model, model_path)
    print(f"✅ {model_type} Model Saved: {model_path}")

## ResNet-50

In [21]:
train_neural_network("ResNet-50", num_epochs=30, batch_size=64, optimizer_type="adam")

🔄 Loading dataset from ResNet-50 features...
✅ Loaded train data from ResNet-50 features: (9246, 4606), Labels: (9246,)
✅ Loaded test data from ResNet-50 features: (2308, 4606), Labels: (2308,)
Epoch [1/30], Loss: 1.5986, Train Accuracy: 57.06%, Test Accuracy: 63.69%
Epoch [2/30], Loss: 1.4987, Train Accuracy: 66.66%, Test Accuracy: 67.85%
Epoch [3/30], Loss: 1.4832, Train Accuracy: 68.23%, Test Accuracy: 65.03%
Epoch [4/30], Loss: 1.4562, Train Accuracy: 70.86%, Test Accuracy: 69.97%
Epoch [5/30], Loss: 1.4325, Train Accuracy: 73.04%, Test Accuracy: 76.39%
Epoch [6/30], Loss: 1.4075, Train Accuracy: 75.70%, Test Accuracy: 74.70%
Epoch [7/30], Loss: 1.3993, Train Accuracy: 76.56%, Test Accuracy: 78.16%
Epoch [8/30], Loss: 1.3824, Train Accuracy: 78.36%, Test Accuracy: 75.87%
Epoch [9/30], Loss: 1.3790, Train Accuracy: 78.49%, Test Accuracy: 75.87%
Epoch [10/30], Loss: 1.3797, Train Accuracy: 78.50%, Test Accuracy: 78.81%
Epoch [11/30], Loss: 1.3769, Train Accuracy: 78.76%, Test Accurac

HFFClassifier(
  (fc1): Linear(in_features=4606, out_features=512, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=7, bias=True)
  (softmax): Softmax(dim=1)
)

In [25]:
train_other_models("ResNet-50", "random_forest")

🔄 Loading dataset from ResNet-50 features...
✅ Loaded train data from ResNet-50 features: (9246, 4606), Labels: (9246,)
✅ Loaded test data from ResNet-50 features: (2308, 4606), Labels: (2308,)
🔄 Training random_forest model on ResNet-50 features...
✅ random_forest Model Accuracy on ResNet-50:
   - Train Accuracy: 100.00%
   - Test Accuracy: 89.43%
✅ random_forest Model Saved: models/hff_random_forest_ResNet-50.pkl


In [26]:
train_other_models("ResNet-50", "svm")

🔄 Loading dataset from ResNet-50 features...
✅ Loaded train data from ResNet-50 features: (9246, 4606), Labels: (9246,)
✅ Loaded test data from ResNet-50 features: (2308, 4606), Labels: (2308,)
🔄 Training svm model on ResNet-50 features...
✅ svm Model Accuracy on ResNet-50:
   - Train Accuracy: 92.43%
   - Test Accuracy: 88.78%
✅ svm Model Saved: models/hff_svm_ResNet-50.pkl


In [27]:
train_other_models("ResNet-50", "logistic_regression")

🔄 Loading dataset from ResNet-50 features...
✅ Loaded train data from ResNet-50 features: (9246, 4606), Labels: (9246,)
✅ Loaded test data from ResNet-50 features: (2308, 4606), Labels: (2308,)
🔄 Training logistic_regression model on ResNet-50 features...
✅ logistic_regression Model Accuracy on ResNet-50:
   - Train Accuracy: 100.00%
   - Test Accuracy: 91.16%
✅ logistic_regression Model Saved: models/hff_logistic_regression_ResNet-50.pkl


In [29]:
# train_other_models("ResNet-50", "gradient_boosting")  

## ResNeSt-50

In [30]:
train_neural_network("ResNeSt-50", num_epochs=30, batch_size=64, optimizer_type="adam")

🔄 Loading dataset from ResNeSt-50 features...
✅ Loaded train data from ResNeSt-50 features: (9246, 4606), Labels: (9246,)
✅ Loaded test data from ResNeSt-50 features: (2308, 4606), Labels: (2308,)
Epoch [1/30], Loss: 1.5504, Train Accuracy: 61.91%, Test Accuracy: 70.28%
Epoch [2/30], Loss: 1.4149, Train Accuracy: 75.26%, Test Accuracy: 77.56%
Epoch [3/30], Loss: 1.3645, Train Accuracy: 80.05%, Test Accuracy: 80.68%
Epoch [4/30], Loss: 1.3554, Train Accuracy: 80.92%, Test Accuracy: 81.85%
Epoch [5/30], Loss: 1.3237, Train Accuracy: 84.25%, Test Accuracy: 78.86%
Epoch [6/30], Loss: 1.3229, Train Accuracy: 84.21%, Test Accuracy: 81.93%
Epoch [7/30], Loss: 1.3017, Train Accuracy: 86.35%, Test Accuracy: 79.90%
Epoch [8/30], Loss: 1.2937, Train Accuracy: 87.10%, Test Accuracy: 85.92%
Epoch [9/30], Loss: 1.2831, Train Accuracy: 88.05%, Test Accuracy: 80.16%
Epoch [10/30], Loss: 1.2716, Train Accuracy: 89.43%, Test Accuracy: 87.52%
Epoch [11/30], Loss: 1.2760, Train Accuracy: 88.85%, Test Accu

HFFClassifier(
  (fc1): Linear(in_features=4606, out_features=512, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=7, bias=True)
  (softmax): Softmax(dim=1)
)

In [31]:
train_other_models("ResNeSt-50", "random_forest")

🔄 Loading dataset from ResNeSt-50 features...
✅ Loaded train data from ResNeSt-50 features: (9246, 4606), Labels: (9246,)
✅ Loaded test data from ResNeSt-50 features: (2308, 4606), Labels: (2308,)
🔄 Training random_forest model on ResNeSt-50 features...
✅ random_forest Model Accuracy on ResNeSt-50:
   - Train Accuracy: 100.00%
   - Test Accuracy: 88.99%
✅ random_forest Model Saved: models/hff_random_forest_ResNeSt-50.pkl


In [32]:
train_other_models("ResNeSt-50", "svm")

🔄 Loading dataset from ResNeSt-50 features...
✅ Loaded train data from ResNeSt-50 features: (9246, 4606), Labels: (9246,)
✅ Loaded test data from ResNeSt-50 features: (2308, 4606), Labels: (2308,)
🔄 Training svm model on ResNeSt-50 features...
✅ svm Model Accuracy on ResNeSt-50:
   - Train Accuracy: 92.06%
   - Test Accuracy: 89.04%
✅ svm Model Saved: models/hff_svm_ResNeSt-50.pkl


In [33]:
train_other_models("ResNeSt-50", "logistic_regression")

🔄 Loading dataset from ResNeSt-50 features...
✅ Loaded train data from ResNeSt-50 features: (9246, 4606), Labels: (9246,)
✅ Loaded test data from ResNeSt-50 features: (2308, 4606), Labels: (2308,)
🔄 Training logistic_regression model on ResNeSt-50 features...
✅ logistic_regression Model Accuracy on ResNeSt-50:
   - Train Accuracy: 100.00%
   - Test Accuracy: 92.50%
✅ logistic_regression Model Saved: models/hff_logistic_regression_ResNeSt-50.pkl


In [34]:
# train_other_models("ResNeSt-50", "gradient_boosting")

## ResNeXt-50

In [35]:
train_neural_network("ResNeXt-50", num_epochs=30, batch_size=64, optimizer_type="adam")

🔄 Loading dataset from ResNeXt-50 features...
✅ Loaded train data from ResNeXt-50 features: (9246, 4606), Labels: (9246,)
✅ Loaded test data from ResNeXt-50 features: (2308, 4606), Labels: (2308,)
Epoch [1/30], Loss: 1.6238, Train Accuracy: 54.54%, Test Accuracy: 61.70%
Epoch [2/30], Loss: 1.4950, Train Accuracy: 67.26%, Test Accuracy: 68.11%
Epoch [3/30], Loss: 1.4502, Train Accuracy: 71.65%, Test Accuracy: 73.27%
Epoch [4/30], Loss: 1.4097, Train Accuracy: 75.79%, Test Accuracy: 78.12%
Epoch [5/30], Loss: 1.3834, Train Accuracy: 78.23%, Test Accuracy: 78.38%
Epoch [6/30], Loss: 1.3663, Train Accuracy: 80.15%, Test Accuracy: 78.38%
Epoch [7/30], Loss: 1.3578, Train Accuracy: 80.76%, Test Accuracy: 79.55%
Epoch [8/30], Loss: 1.3611, Train Accuracy: 80.43%, Test Accuracy: 77.99%
Epoch [9/30], Loss: 1.3492, Train Accuracy: 81.71%, Test Accuracy: 79.51%
Epoch [10/30], Loss: 1.3446, Train Accuracy: 82.15%, Test Accuracy: 79.90%
Epoch [11/30], Loss: 1.3414, Train Accuracy: 82.42%, Test Accu

HFFClassifier(
  (fc1): Linear(in_features=4606, out_features=512, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=7, bias=True)
  (softmax): Softmax(dim=1)
)

In [36]:
train_other_models("ResNeXt-50", "random_forest")

🔄 Loading dataset from ResNeXt-50 features...
✅ Loaded train data from ResNeXt-50 features: (9246, 4606), Labels: (9246,)
✅ Loaded test data from ResNeXt-50 features: (2308, 4606), Labels: (2308,)
🔄 Training random_forest model on ResNeXt-50 features...
✅ random_forest Model Accuracy on ResNeXt-50:
   - Train Accuracy: 100.00%
   - Test Accuracy: 89.12%
✅ random_forest Model Saved: models/hff_random_forest_ResNeXt-50.pkl


In [37]:
train_other_models("ResNeXt-50", "svm")

🔄 Loading dataset from ResNeXt-50 features...
✅ Loaded train data from ResNeXt-50 features: (9246, 4606), Labels: (9246,)
✅ Loaded test data from ResNeXt-50 features: (2308, 4606), Labels: (2308,)
🔄 Training svm model on ResNeXt-50 features...
✅ svm Model Accuracy on ResNeXt-50:
   - Train Accuracy: 94.67%
   - Test Accuracy: 88.91%
✅ svm Model Saved: models/hff_svm_ResNeXt-50.pkl


In [38]:
train_other_models("ResNeXt-50", "logistic_regression")

🔄 Loading dataset from ResNeXt-50 features...
✅ Loaded train data from ResNeXt-50 features: (9246, 4606), Labels: (9246,)
✅ Loaded test data from ResNeXt-50 features: (2308, 4606), Labels: (2308,)
🔄 Training logistic_regression model on ResNeXt-50 features...
✅ logistic_regression Model Accuracy on ResNeXt-50:
   - Train Accuracy: 100.00%
   - Test Accuracy: 89.90%
✅ logistic_regression Model Saved: models/hff_logistic_regression_ResNeXt-50.pkl


In [39]:
# train_other_models("ResNeXt-50", "gradient_boosting")

## SE-ResNet-50

In [40]:
train_neural_network("SE-ResNet-50", num_epochs=30, batch_size=64, optimizer_type="adam")

🔄 Loading dataset from SE-ResNet-50 features...
✅ Loaded train data from SE-ResNet-50 features: (9246, 4606), Labels: (9246,)
✅ Loaded test data from SE-ResNet-50 features: (2308, 4606), Labels: (2308,)
Epoch [1/30], Loss: 1.6381, Train Accuracy: 53.15%, Test Accuracy: 63.91%
Epoch [2/30], Loss: 1.4953, Train Accuracy: 67.42%, Test Accuracy: 69.84%
Epoch [3/30], Loss: 1.4486, Train Accuracy: 71.88%, Test Accuracy: 68.46%
Epoch [4/30], Loss: 1.4197, Train Accuracy: 74.67%, Test Accuracy: 73.44%
Epoch [5/30], Loss: 1.3838, Train Accuracy: 78.30%, Test Accuracy: 79.33%
Epoch [6/30], Loss: 1.3508, Train Accuracy: 81.55%, Test Accuracy: 80.94%
Epoch [7/30], Loss: 1.3315, Train Accuracy: 83.52%, Test Accuracy: 83.93%
Epoch [8/30], Loss: 1.3141, Train Accuracy: 85.25%, Test Accuracy: 84.32%
Epoch [9/30], Loss: 1.3141, Train Accuracy: 85.15%, Test Accuracy: 83.28%
Epoch [10/30], Loss: 1.3103, Train Accuracy: 85.50%, Test Accuracy: 81.41%
Epoch [11/30], Loss: 1.3063, Train Accuracy: 86.02%, Tes

HFFClassifier(
  (fc1): Linear(in_features=4606, out_features=512, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=7, bias=True)
  (softmax): Softmax(dim=1)
)

In [41]:
train_other_models("SE-ResNet-50", "random_forest")

🔄 Loading dataset from SE-ResNet-50 features...
✅ Loaded train data from SE-ResNet-50 features: (9246, 4606), Labels: (9246,)
✅ Loaded test data from SE-ResNet-50 features: (2308, 4606), Labels: (2308,)
🔄 Training random_forest model on SE-ResNet-50 features...
✅ random_forest Model Accuracy on SE-ResNet-50:
   - Train Accuracy: 100.00%
   - Test Accuracy: 88.21%
✅ random_forest Model Saved: models/hff_random_forest_SE-ResNet-50.pkl


In [42]:
train_other_models("SE-ResNet-50", "svm")

🔄 Loading dataset from SE-ResNet-50 features...
✅ Loaded train data from SE-ResNet-50 features: (9246, 4606), Labels: (9246,)
✅ Loaded test data from SE-ResNet-50 features: (2308, 4606), Labels: (2308,)
🔄 Training svm model on SE-ResNet-50 features...
✅ svm Model Accuracy on SE-ResNet-50:
   - Train Accuracy: 93.02%
   - Test Accuracy: 86.48%
✅ svm Model Saved: models/hff_svm_SE-ResNet-50.pkl


In [43]:
train_other_models("SE-ResNet-50", "logistic_regression")

🔄 Loading dataset from SE-ResNet-50 features...
✅ Loaded train data from SE-ResNet-50 features: (9246, 4606), Labels: (9246,)
✅ Loaded test data from SE-ResNet-50 features: (2308, 4606), Labels: (2308,)
🔄 Training logistic_regression model on SE-ResNet-50 features...
✅ logistic_regression Model Accuracy on SE-ResNet-50:
   - Train Accuracy: 99.99%
   - Test Accuracy: 88.21%
✅ logistic_regression Model Saved: models/hff_logistic_regression_SE-ResNet-50.pkl


In [44]:
# train_other_models("SE-ResNet-50", "gradient_boosting")

## DenseNet

In [45]:
train_neural_network("DenseNet-121", num_epochs=30, batch_size=64, optimizer_type="adam")

🔄 Loading dataset from DenseNet-121 features...
✅ Loaded train data from DenseNet-121 features: (9246, 52734), Labels: (9246,)
✅ Loaded test data from DenseNet-121 features: (2308, 52734), Labels: (2308,)
Epoch [1/30], Loss: 1.7395, Train Accuracy: 42.47%, Test Accuracy: 46.53%
Epoch [2/30], Loss: 1.7037, Train Accuracy: 46.15%, Test Accuracy: 47.83%
Epoch [3/30], Loss: 1.7225, Train Accuracy: 44.32%, Test Accuracy: 37.31%
Epoch [4/30], Loss: 1.8059, Train Accuracy: 35.98%, Test Accuracy: 37.44%
Epoch [5/30], Loss: 1.7504, Train Accuracy: 41.48%, Test Accuracy: 46.62%
Epoch [6/30], Loss: 1.7055, Train Accuracy: 46.03%, Test Accuracy: 49.13%
Epoch [7/30], Loss: 1.7233, Train Accuracy: 44.24%, Test Accuracy: 45.75%
Epoch [8/30], Loss: 1.7570, Train Accuracy: 40.81%, Test Accuracy: 46.66%
Epoch [9/30], Loss: 1.7674, Train Accuracy: 39.74%, Test Accuracy: 44.24%
Epoch [10/30], Loss: 1.7090, Train Accuracy: 45.62%, Test Accuracy: 43.63%
Epoch [11/30], Loss: 1.7155, Train Accuracy: 44.94%, T

HFFClassifier(
  (fc1): Linear(in_features=52734, out_features=512, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=7, bias=True)
  (softmax): Softmax(dim=1)
)

In [46]:
train_other_models("DenseNet-121", "random_forest")

🔄 Loading dataset from DenseNet-121 features...
✅ Loaded train data from DenseNet-121 features: (9246, 52734), Labels: (9246,)
✅ Loaded test data from DenseNet-121 features: (2308, 52734), Labels: (2308,)
🔄 Training random_forest model on DenseNet-121 features...
✅ random_forest Model Accuracy on DenseNet-121:
   - Train Accuracy: 100.00%
   - Test Accuracy: 84.14%
✅ random_forest Model Saved: models/hff_random_forest_DenseNet-121.pkl


In [ ]:
# train_other_models("DenseNet-121", "logistic_regression")

## Gradient Boosting

In [ ]:
# train_other_models("DenseNet-121", "gradient_boosting")

In [48]:
train_other_models("ResNeSt-50", "gradient_boosting")

🔄 Loading dataset from ResNeSt-50 features...
✅ Loaded train data from ResNeSt-50 features: (9246, 4606), Labels: (9246,)
✅ Loaded test data from ResNeSt-50 features: (2308, 4606), Labels: (2308,)
🔄 Training gradient_boosting model on ResNeSt-50 features...
✅ gradient_boosting Model Accuracy on ResNeSt-50:
   - Train Accuracy: 97.66%
   - Test Accuracy: 88.43%
✅ gradient_boosting Model Saved: models/hff_gradient_boosting_ResNeSt-50.pkl


In [49]:
train_other_models("ResNet-50", "gradient_boosting")

🔄 Loading dataset from ResNet-50 features...
✅ Loaded train data from ResNet-50 features: (9246, 4606), Labels: (9246,)
✅ Loaded test data from ResNet-50 features: (2308, 4606), Labels: (2308,)
🔄 Training gradient_boosting model on ResNet-50 features...
✅ gradient_boosting Model Accuracy on ResNet-50:
   - Train Accuracy: 98.25%
   - Test Accuracy: 89.38%
✅ gradient_boosting Model Saved: models/hff_gradient_boosting_ResNet-50.pkl


In [50]:
train_other_models("ResNeXt-50", "gradient_boosting")

🔄 Loading dataset from ResNeXt-50 features...
✅ Loaded train data from ResNeXt-50 features: (9246, 4606), Labels: (9246,)
✅ Loaded test data from ResNeXt-50 features: (2308, 4606), Labels: (2308,)
🔄 Training gradient_boosting model on ResNeXt-50 features...
✅ gradient_boosting Model Accuracy on ResNeXt-50:
   - Train Accuracy: 98.09%
   - Test Accuracy: 88.00%
✅ gradient_boosting Model Saved: models/hff_gradient_boosting_ResNeXt-50.pkl


In [51]:
train_other_models("SE-ResNet-50", "gradient_boosting")

🔄 Loading dataset from SE-ResNet-50 features...
✅ Loaded train data from SE-ResNet-50 features: (9246, 4606), Labels: (9246,)
✅ Loaded test data from SE-ResNet-50 features: (2308, 4606), Labels: (2308,)
🔄 Training gradient_boosting model on SE-ResNet-50 features...
✅ gradient_boosting Model Accuracy on SE-ResNet-50:
   - Train Accuracy: 98.01%
   - Test Accuracy: 88.52%
✅ gradient_boosting Model Saved: models/hff_gradient_boosting_SE-ResNet-50.pkl
